# Neural Network using tensorflow
for this model whe used some information from "https://machinelearningmastery.com/tensorflow-tutorial-deep-learning-with-tf-keras/"

In [1]:
import tensorflow.keras as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam



In [2]:
complete_df = pd.read_csv('data/AmesHousingPreprocessed.csv')
complete_df = complete_df.drop('Unnamed: 0', axis=1)
display(complete_df)

,MS Zoning_C (all),MS Zoning_FV,MS Zoning_I (all),MS Zoning_RH,MS Zoning_RL,MS Zoning_RM,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,...,Year Remod/Add,Full Bath,Year Built,1st Flr SF,Total Bsmt SF,Garage Area,Garage Cars,Gr Liv Area,Overall Qual,SalePrice
0,0,0,0,0,1,0,0,0,0,0,...,-1.155886,-1.019607,-0.368243,1.281557,0.065626,0.258714,0.309465,0.313663,-0.061812,12.278393
1,0,0,0,1,0,0,0,0,0,0,...,-1.107938,-1.019607,-0.335146,-0.675537,-0.384244,1.197200,-1.003952,-1.192617,-0.771063,11.561716
2,0,0,0,0,1,0,0,0,0,0,...,-1.251781,-1.019607,-0.434438,0.439492,0.631372,-0.744814,-1.003952,-0.334434,-0.061812,12.055250
3,0,0,0,0,1,0,0,0,0,0,...,-0.772303,0.794069,-0.103465,2.450663,2.405858,0.230839,0.309465,1.213467,0.647439,12.404924
4,0,0,0,0,1,0,0,0,0,0,...,0.666134,0.794069,0.856357,-0.593133,-0.279728,0.045000,0.309465,0.260151,-0.771063,12.154253
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2898,0,0,0,0,1,0,0,0,0,0,...,-0.005137,-1.019607,0.426092,-0.399998,-0.109323,0.537472,0.309465,-0.980548,-0.061812,11.867097
2899,0,0,0,0,1,0,0,0,0,0,...,-0.053085,-1.019607,0.392995,-0.660086,-0.425141,0.054292,0.309465,-1.180725,-0.771063,11.782953
2900,0,0,0,0,1,0,0,0,0,0,...,0.378446,-1.019607,0.690871,-0.484977,-0.316081,-2.194355,-2.317368,-1.045953,-0.771063,11.790557
2901,0,0,0,0,1,0,0,0,0,0,...,-0.436668,-1.019607,0.095119,0.593999,0.767696,-0.252342,0.309465,-0.215517,-0.771063,12.043554


In [3]:
# split data into input and target
target = complete_df['SalePrice']
input_df = complete_df.drop('SalePrice', axis=1)

display(input_df.head())

,MS Zoning_C (all),MS Zoning_FV,MS Zoning_I (all),MS Zoning_RH,MS Zoning_RL,MS Zoning_RM,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,...,Garage Yr Blt,Year Remod/Add,Full Bath,Year Built,1st Flr SF,Total Bsmt SF,Garage Area,Garage Cars,Gr Liv Area,Overall Qual
0,0,0,0,0,1,0,0,0,0,0,...,-0.730498,-1.155886,-1.019607,-0.368243,1.281557,0.065626,0.258714,0.309465,0.313663,-0.061812
1,0,0,0,1,0,0,0,0,0,0,...,-0.689605,-1.107938,-1.019607,-0.335146,-0.675537,-0.384244,1.197200,-1.003952,-1.192617,-0.771063
2,0,0,0,0,1,0,0,0,0,0,...,-0.812285,-1.251781,-1.019607,-0.434438,0.439492,0.631372,-0.744814,-1.003952,-0.334434,-0.061812
3,0,0,0,0,1,0,0,0,0,0,...,-0.403350,-0.772303,0.794069,-0.103465,2.450663,2.405858,0.230839,0.309465,1.213467,0.647439
4,0,0,0,0,1,0,0,0,0,0,...,0.782561,0.666134,0.794069,0.856357,-0.593133,-0.279728,0.045000,0.309465,0.260151,-0.771063


In [4]:
# convert the pandas dataframes to numpy ndarrays
X_np = input_df.to_numpy()
y_np = target.to_numpy()

# split the data into 70% training and 30% testing
X_train, X_test, y_train, y_test = train_test_split(X_np, y_np, train_size=0.7, random_state=1265599650)

In [5]:
# compile the model, try different metric (mean_absolute_percentage_error) later
# link: "https://www.tensorflow.org/api_docs/python/tf/keras/metrics/RootMeanSquaredError"
# use keras API
n_features = input_df.shape[1]

model = tf.Sequential([
    Dense(1, input_shape=(n_features,)),
])

# https://medium.com/mdr-inc/from-sgd-to-adam-c9fce513c4bb 
model.compile(optimizer='Adam', loss=tf.metrics.mean_squared_error, metrics=['mean_absolute_percentage_error'])

In [6]:
# fit the model
# we took a batch size of 32, because it is a common used batch size 
# according to the following site:
# https://machinelearningmastery.com/how-to-control-the-speed-and-stability-of-training-neural-networks-with-gradient-descent-batch-size/
model.fit(X_train, y_train,
          batch_size=32, epochs=100,
          validation_data=(X_test, y_test))

Epoch 1/100
64/64 [==============================] - 1s 7ms/step - loss: 125.3083 - mean_absolute_percentage_error: 92.5620 - val_loss: 116.8666 - val_mean_absolute_percentage_error: 89.4048
Epoch 2/100
64/64 [==============================] - 0s 3ms/step - loss: 110.1805 - mean_absolute_percentage_error: 86.8411 - val_loss: 102.4754 - val_mean_absolute_percentage_error: 83.7684
Epoch 3/100
64/64 [==============================] - 0s 3ms/step - loss: 96.4846 - mean_absolute_percentage_error: 81.3055 - val_loss: 89.4967 - val_mean_absolute_percentage_error: 78.3213
Epoch 4/100
64/64 [==============================] - 0s 3ms/step - loss: 84.1300 - mean_absolute_percentage_error: 75.9313 - val_loss: 77.8492 - val_mean_absolute_percentage_error: 73.0582
Epoch 5/100
64/64 [==============================] - 0s 3ms/step - loss: 73.0422 - mean_absolute_percentage_error: 70.7577 - val_loss: 67.4266 - val_mean_absolute_percentage_error: 67.9818
Epoch 6/100
64/64 [==============================] 

Epoch 45/100
64/64 [==============================] - 0s 3ms/step - loss: 0.6444 - mean_absolute_percentage_error: 4.9630 - val_loss: 0.6118 - val_mean_absolute_percentage_error: 4.8306
Epoch 46/100
64/64 [==============================] - 0s 3ms/step - loss: 0.6341 - mean_absolute_percentage_error: 4.9262 - val_loss: 0.6028 - val_mean_absolute_percentage_error: 4.7989
Epoch 47/100
64/64 [==============================] - 0s 3ms/step - loss: 0.6238 - mean_absolute_percentage_error: 4.8944 - val_loss: 0.5938 - val_mean_absolute_percentage_error: 4.7653
Epoch 48/100
64/64 [==============================] - 0s 3ms/step - loss: 0.6138 - mean_absolute_percentage_error: 4.8527 - val_loss: 0.5850 - val_mean_absolute_percentage_error: 4.7316
Epoch 49/100
64/64 [==============================] - 0s 3ms/step - loss: 0.6040 - mean_absolute_percentage_error: 4.8193 - val_loss: 0.5763 - val_mean_absolute_percentage_error: 4.6994
Epoch 50/100
64/64 [==============================] - ETA: 0s - loss: 

Epoch 89/100
64/64 [==============================] - 0s 3ms/step - loss: 0.3498 - mean_absolute_percentage_error: 3.6562 - val_loss: 0.3476 - val_mean_absolute_percentage_error: 3.6661
Epoch 90/100
64/64 [==============================] - 0s 3ms/step - loss: 0.3447 - mean_absolute_percentage_error: 3.6330 - val_loss: 0.3437 - val_mean_absolute_percentage_error: 3.6446
Epoch 91/100
64/64 [==============================] - 0s 3ms/step - loss: 0.3407 - mean_absolute_percentage_error: 3.6070 - val_loss: 0.3391 - val_mean_absolute_percentage_error: 3.6129
Epoch 92/100
64/64 [==============================] - 0s 3ms/step - loss: 0.3363 - mean_absolute_percentage_error: 3.5875 - val_loss: 0.3353 - val_mean_absolute_percentage_error: 3.5943
Epoch 93/100
64/64 [==============================] - 0s 3ms/step - loss: 0.3320 - mean_absolute_percentage_error: 3.5530 - val_loss: 0.3308 - val_mean_absolute_percentage_error: 3.5644
Epoch 94/100
64/64 [==============================] - 0s 3ms/step - lo

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1)                 84        
                                                                 
Total params: 84
Trainable params: 84
Non-trainable params: 0
_________________________________________________________________


### Model 2

In [8]:
#### verebeterpunten:
# validation data gebruiken
# k folds
# andere metrics voor aflezen
# adam vs andere andere sgd methods testen
# hidden layers toevoegen
# aantal nodes per layer aanpassen
# batchsize aanpassen (32, 64 of 128)
# aantal epochs 
# onze loss plotten